In [103]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
%autoreload 2
import math
import numpy as np

from BoseHubbardHamiltonian import BoseHubbardHamiltonian

from qiskit.quantum_info import Pauli, Operator
from qiskit.quantum_info import SparsePauliOp

from symmer.operators import PauliwordOp, QuantumState
from symmer.projection import QubitTapering 

import rustworkx as rx
from qiskit_nature.second_q.hamiltonians.lattices import (
    BoundaryCondition,
    HyperCubicLattice,
    Lattice,
    LatticeDrawStyle,
    LineLattice,
    SquareLattice,
    TriangularLattice,
)

from symmer.operators import IndependentOp
from scipy.linalg import eigh
import scipy.sparse

In [105]:
#line lattice 
num_nodes = 2 #number of sites in the lattice 
boundary_condition = BoundaryCondition.OPEN #open lattice
line_lattice = LineLattice(num_nodes=num_nodes, boundary_condition=boundary_condition)

In [106]:
#triangular lattice 
rows = 4
cols = 3
boundary_condition = BoundaryCondition.PERIODIC

triangular_lattice = TriangularLattice(rows=rows, cols=cols, boundary_condition=boundary_condition)

In [107]:
#general lattice
graph = rx.PyGraph(multigraph=False)  # multigraph shoud be False
graph.add_nodes_from(range(4))
weighted_edge_list = [
    (1, 0, 1.0), 
    (1, 2, 1.0), 
    (1, 3, 1.0), 
]
graph.add_edges_from(weighted_edge_list)

# make a lattice
general_lattice = Lattice(graph)
set(general_lattice.graph.weighted_edge_list())

{(1, 0, 1.0), (1, 2, 1.0), (1, 3, 1.0)}

In [108]:
#Hamiltonian of the system given the number of sites in the lattice and the number of qubits 
H_instance = BoseHubbardHamiltonian(4, 2, general_lattice)
H_q = H_instance.get_H(1,1)

In [109]:
#Hamiltonian to be tapered
H = PauliwordOp.from_qiskit(H_q)
print(H)

 0.933+0.000j IXIXIIII +
 0.483+0.000j IXXXIIII +
 0.483+0.000j IXYYIIII +
-0.250+0.000j IXZXIIII +
 0.933+0.000j IYIYIIII +
-0.483+0.000j IYXYIIII +
 0.483+0.000j IYYXIIII +
-0.250+0.000j IYZYIIII +
 0.483+0.000j XXIXIIII +
 0.250+0.000j XXXXIIII +
 0.250+0.000j XXYYIIII +
-0.129+0.000j XXZXIIII +
-0.483+0.000j XYIYIIII +
 0.250+0.000j XYXYIIII +
-0.250+0.000j XYYXIIII +
 0.129+0.000j XYZYIIII +
 0.483+0.000j YXIYIIII +
-0.250+0.000j YXXYIIII +
 0.250+0.000j YXYXIIII +
-0.129+0.000j YXZYIIII +
 0.483+0.000j YYIXIIII +
 0.250+0.000j YYXXIIII +
 0.250+0.000j YYYYIIII +
-0.129+0.000j YYZXIIII +
-0.250+0.000j ZXIXIIII +
-0.129+0.000j ZXXXIIII +
-0.129+0.000j ZXYYIIII +
 0.067+0.000j ZXZXIIII +
-0.250+0.000j ZYIYIIII +
 0.129+0.000j ZYXYIIII +
-0.129+0.000j ZYYXIIII +
 0.067+0.000j ZYZYIIII +
 0.933+0.000j IIIXIXII +
 0.483+0.000j IIIXXXII +
 0.483+0.000j IIIXYYII +
-0.250+0.000j IIIXZXII +
 0.933+0.000j IIIYIYII +
-0.483+0.000j IIIYXYII +
 0.483+0.000j IIIYYXII +
-0.250+0.000j IIIYZYII +


In [110]:
#IndipendentOp represents algebraically independent sets of Pauli operators for stabilizer manipulation/projections
IndependentOp.symmetry_generators(H, commuting_override=True)

1 IZIZIZIZ

In [111]:
taper_hamiltonian = QubitTapering(H)
taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis()

-1 IZIIIIII

In [112]:
print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand may be rotated onto the single-qubit Pauli operators\n')
print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
print('\nvia a sequence of rotations e^{i pi/4 R} where\n')
for index, (rot, angle) in enumerate(taper_hamiltonian.stabilizers.stabilizer_rotations):
    print(f'R_{index} = {rot}')

We are able to taper 1 qubits from the Hamiltonian.

The symmetry generators are

1 IZIZIZIZ

and may be rotated onto the single-qubit Pauli operators

-1 IZIIIIII

via a sequence of rotations e^{i pi/4 R} where

R_0 =  1.000+0.000j IYIZIZIZ
R_1 =  1.000+0.000j IYIIIIII


In [113]:
sec_array = [1]
ham_tap_p = taper_hamiltonian.taper_it(sector=sec_array)
print('Tapered Hamiltonian:\n')
print(ham_tap_p) #sector = +1

Tapered Hamiltonian:

 8.000+0.000j IIIIIII +
 1.000+0.000j IIIIIIZ +
-1.000+0.000j IIIIIZI +
-2.000+0.000j IIIIIZZ +
 1.000+0.000j IIIIZII +
-1.000+0.000j IIIZIII +
-2.000+0.000j IIIZZII +
 1.000+0.000j IIZIIII +
 1.000-0.000j IIZIZIZ +
-1.000+0.000j IZIIIII +
-2.000+0.000j IZZIIII +
-1.000+0.000j ZIIIIII +
-2.000+0.000j ZIZIZIZ +
-0.933+0.000j IIXIIII +
 0.933+0.000j IIXIZIZ +
 0.250-0.000j IZXIIII +
-0.250+0.000j IZXIZIZ +
 0.250-0.000j ZIXIIII +
-0.250+0.000j ZIXIZIZ +
-0.067+0.000j ZZXIIII +
 0.067+0.000j ZZXIZIZ +
 0.933+0.000j IIXIIIX +
-0.250+0.000j IIXIIZX +
 0.933+0.000j IIYIIIY +
-0.250+0.000j IIYIIZY +
-0.250+0.000j IZXIIIX +
 0.067+0.000j IZXIIZX +
-0.250+0.000j IZYIIIY +
 0.067+0.000j IZYIIZY +
 0.483+0.000j IIXIIXX +
 0.483+0.000j IIXIIYY +
-0.483+0.000j IIYIIXY +
 0.483+0.000j IIYIIYX +
-0.129+0.000j IZXIIXX +
-0.129+0.000j IZXIIYY +
 0.129+0.000j IZYIIXY +
-0.129+0.000j IZYIIYX +
 0.933+0.000j IIXIXII +
-0.250+0.000j IIXZXII +
 0.933+0.000j IIYIYII +
-0.250+0.000j IIYZ

In [114]:
sec_array = [-1]
ham_tap_n = taper_hamiltonian.taper_it(sector=sec_array)
print('Tapered Hamiltonian:\n')
print(ham_tap_n) #sector = -1

Tapered Hamiltonian:

 8.000+0.000j IIIIIII +
 1.000+0.000j IIIIIIZ +
-1.000+0.000j IIIIIZI +
-2.000+0.000j IIIIIZZ +
 1.000+0.000j IIIIZII +
-1.000+0.000j IIIZIII +
-2.000+0.000j IIIZZII +
 1.000+0.000j IIZIIII +
-1.000+0.000j IIZIZIZ +
-1.000+0.000j IZIIIII +
-2.000+0.000j IZZIIII +
-1.000+0.000j ZIIIIII +
 2.000+0.000j ZIZIZIZ +
 0.933+0.000j IIXIIII +
 0.933+0.000j IIXIZIZ +
-0.250+0.000j IZXIIII +
-0.250+0.000j IZXIZIZ +
-0.250+0.000j ZIXIIII +
-0.250+0.000j ZIXIZIZ +
 0.067+0.000j ZZXIIII +
 0.067+0.000j ZZXIZIZ +
 0.933+0.000j IIXIIIX +
-0.250+0.000j IIXIIZX +
 0.933+0.000j IIYIIIY +
-0.250+0.000j IIYIIZY +
-0.250+0.000j IZXIIIX +
 0.067+0.000j IZXIIZX +
-0.250+0.000j IZYIIIY +
 0.067+0.000j IZYIIZY +
 0.483+0.000j IIXIIXX +
 0.483+0.000j IIXIIYY +
-0.483+0.000j IIYIIXY +
 0.483+0.000j IIYIIYX +
-0.129+0.000j IZXIIXX +
-0.129+0.000j IZXIIYY +
 0.129+0.000j IZYIIXY +
-0.129+0.000j IZYIIYX +
 0.933+0.000j IIXIXII +
-0.250+0.000j IIXZXII +
 0.933+0.000j IIYIYII +
-0.250+0.000j IIYZ

In [115]:
H_sparse_matrix = H.to_sparse_matrix
H_array = H_sparse_matrix.toarray()

In [116]:
np.set_printoptions(precision=20)
eigh(H_array, eigvals_only=True)

array([-1.3434805042347402e+00, -1.2348212465647406e+00,
       -1.1219850774341740e+00, -9.9229715574397492e-01,
       -9.9229715574396526e-01, -7.3419927072351976e-01,
       -7.3419927072351854e-01, -6.2327064311461688e-01,
       -6.2293751013262755e-01, -5.1864477592365454e-01,
       -5.1658948432480678e-01, -5.1658948432479457e-01,
       -4.9606030379828669e-01, -4.5256669639651731e-01,
       -4.0327850012002570e-01, -4.0327850012001115e-01,
       -3.9302296558406602e-01, -3.9302296558406224e-01,
       -3.5889894354067209e-01, -9.4670533768350981e-02,
       -9.4670533768349691e-02, -3.6853219343260654e-16,
        3.1086244689504383e-15,  2.0988389884774183e-01,
        2.6794919243112580e-01,  2.7370963224402961e-01,
        5.6661750485099771e-01,  5.6661750485100404e-01,
        5.8117041058936036e-01,  6.7094770303737161e-01,
        6.7094770303737772e-01,  7.2889431446141795e-01,
        7.2889431446142072e-01,  8.0384757729337930e-01,
        1.0376113918159717e+00,

In [117]:
ham_tap_p_sparse_matrix = ham_tap_p.to_sparse_matrix
ham_tap_p_array = ham_tap_p_sparse_matrix.toarray()
eigenvalues_p = eigh(ham_tap_p_array, eigvals_only=True)
np.set_printoptions(precision=20)
print(eigenvalues_p)

[-1.1219850774341642e+00 -9.9229715574398314e-01 -9.9229715574398036e-01
 -7.3419927072352020e-01 -7.3419927072351709e-01 -6.2293751013262078e-01
 -5.1864477592365366e-01 -4.9606030379830751e-01 -9.4670533768347123e-02
 -9.4670533768345416e-02  3.3306690738754696e-15  7.0948713153744784e-15
  2.0988389884773018e-01  5.6661750485099283e-01  5.6661750485100160e-01
  5.8117041058935692e-01  1.0376113918159737e+00  1.1878937398909251e+00
  1.1878937398909319e+00  1.3856512651994557e+00  1.3856512651994679e+00
  1.6183513128878189e+00  1.6183513128878244e+00  1.6411010564593329e+00
  1.9999999999999867e+00  2.2846410310297043e+00  2.4745605063654419e+00
  2.6570769172228346e+00  2.6570769172228386e+00  2.6610849199289075e+00
  2.6610849199289173e+00  2.9132830911090233e+00  2.9516756432412192e+00
  2.9516756432412272e+00  3.0082600542265814e+00  3.0082600542265827e+00
  3.3619868827558834e+00  3.3619868827558874e+00  3.5461244524743423e+00
  3.8730716737590014e+00  3.8730716737590050e+00  4

In [119]:
ham_tap_n_sparse_matrix = ham_tap_n.to_sparse_matrix
ham_tap_n_array = ham_tap_n_sparse_matrix.toarray()
eigenvalues_n = eigh(ham_tap_n_array, eigvals_only=True)
np.set_printoptions(precision=20)
print(eigenvalues_n)

[-1.3434805042347269  -1.23482124656474    -0.6232706431146108
 -0.5165894843248031  -0.5165894843247869  -0.4525666963965115
 -0.40327850012001853 -0.4032785001200167  -0.3930229655840629
 -0.3930229655840624  -0.3588989435406722   0.26794919243112597
  0.2737096322440146   0.6709477030373733   0.6709477030373835
  0.7288943144614219   0.7288943144614276   0.8038475772933785
  1.0872631737502243   1.3990206783777566   1.3990206783777666
  1.420109834125044    1.9814195915667674   2.0000000000000036
  2.0000000000000053   2.18772098257787     2.1877209825778725
  2.2465820250512034   2.313248454466069    2.313248454466073
  2.831549113566957    2.9134660626272164   2.9402276733674664
  3.2382888402572565   3.2585781549807162   3.2585781549807185
  3.4134570499450607   3.4134570499450723   3.483798613767504
  3.483798613767506    3.732050807568881    4.088180504880047
  4.088180504880048    4.5013633464472145   4.702840313069422
  4.702840313069428    5.170538678099642    6.000000000000